# Lesson 2: RAG Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xx"

In [3]:
import utils


import openai
# openai.api_key = utils.get_openai_api_key()


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [5]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [7]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [8]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [9]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'To create your AI portfolio, you should focus on building a collection of projects that demonstrate your skill progression. This means starting with simpler projects and gradually tackling more complex ones as you gain experience. By showcasing a variety of projects, you can highlight your expertise in different areas of AI. Additionally, it is important to ensure that your portfolio reflects the specific skills and technologies that are relevant to the AI job market. This will help you stand out to potential employers and increase your chances of finding the right AI job for you.'

## Feedback functions

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [12]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [13]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [14]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [15]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .


✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [16]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [17]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [18]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [19]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [20]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find part of AI challenging?']

In [21]:
eval_questions.append("How can I be successful in AI?")

In [22]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find part of AI challenging?',
 'How can I be successful in AI?']

In [23]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [24]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a350b017c7f04b22d50b0e0c6f8f72ff,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include l...",-,"{""record_id"": ""record_hash_a350b017c7f04b22d50...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:48:51.948648"", ""...",2023-12-27T04:48:56.934637,1.0,1.000,0.40,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'question': 'What are the keys to b...,4,509,0.000779
1,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f994fa2fe541074004416ff9cbe30ce5,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_f994fa2fe5410740044...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:48:57.332829"", ""...",2023-12-27T04:49:04.618918,1.0,0.925,0.65,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,7,662,0.001045
2,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_4f63776ae4c854de4291064257ceb02a,"""What is the importance of networking in AI?""","""Networking is important in AI because it can ...",-,"{""record_id"": ""record_hash_4f63776ae4c854de429...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:49:05.004926"", ""...",2023-12-27T04:49:11.565054,1.0,0.500,0.10,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,6,537,0.000857
3,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_34631b63bbea527e8fc319a76fc05900,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_34631b63bbea527e8fc...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:49:11.931681"", ""...",2023-12-27T04:49:16.663828,1.0,0.500,0.30,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,4,452,0.000705
4,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f576d439419061b181eeab643b6656e0,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_f576d439419061b181e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-27T04:49:17.026354"", ""...",2023-12-27T04:49:22.255275,1.0,0.400,0.50,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Many successful people d...,[{'args': {'question': 'How can altruism be be...,5,554,0.000878


In [25]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.""",0.40,1.0,1.000
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively with a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.""",0.65,1.0,0.925
2,"""What is the importance of networking in AI?""","""Networking is important in AI because it can help individuals build connections with people who are already in the field. These connections can provide valuable insights, guidance, and support, especially for those who are new to the industry. Networking can also help individuals discover job opportunities and stay updated on the latest trends and developments in AI. By reaching out to others in their network, attending meetups, and conducting informational interviews, individuals can expand their professional connections and potentially find mentors or colleagues who can help them advance in their AI careers.""",0.10,1.0,0.500
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can contribute to a successful career. These habits help individuals move forward while staying healthy and can have a positive impact on their overall well-being and professional growth.""",0.30,1.0,0.500
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by helping others even as one focuses on their own career growth. By aiming to lift others during every step of their own journey, individuals can achieve better outcomes for themselves. This can create a positive reputation and build strong relationships, which can lead to opportunities and support in their career advancement. Additionally, practicing altruism can enhance personal satisfaction and fulfillment, which can contribute to overall well-being and success in one's career.""",0.50,1.0,0.400
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological phenomenon where individuals doubt their own abilities and fear being exposed as a fraud, despite evidence of their success. In the context of AI, newcomers to the field sometimes experience imposter syndrome, questioning whether they truly belong in the AI community and if they are capable of contributing to the field. It is important to address imposter syndrome in order to encourage individuals to continue growing in AI and not let self-doubt hinder their progress.""",0.40,0.9,1.000
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",0.45,1.0,1.000
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",0.40,1.0,1.000
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with evolving technologies, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating with stakeho

In [26]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
App_1,0.4,0.99,0.790625,4.727273,0.000869


In [27]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…